## लॉजिस्टिक रिग्रेशन मॉडल बनाएं - पाठ 4

![लॉजिस्टिक बनाम लीनियर रिग्रेशन इन्फोग्राफिक](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.hi.png)

#### **[पाठ-पूर्व क्विज़](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### परिचय

रिग्रेशन पर इस अंतिम पाठ में, जो कि *क्लासिक* मशीन लर्निंग तकनीकों में से एक है, हम लॉजिस्टिक रिग्रेशन पर नज़र डालेंगे। इस तकनीक का उपयोग आप पैटर्न खोजने और बाइनरी श्रेणियों की भविष्यवाणी करने के लिए करेंगे। क्या यह कैंडी चॉकलेट है या नहीं? क्या यह बीमारी संक्रामक है या नहीं? क्या यह ग्राहक इस उत्पाद को चुनेगा या नहीं?

इस पाठ में, आप सीखेंगे:

-   लॉजिस्टिक रिग्रेशन के लिए तकनीकें

✅ इस प्रकार के रिग्रेशन के साथ काम करने की अपनी समझ को इस [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott) में गहराई से बढ़ाएं।

## पूर्वापेक्षा

कद्दू के डेटा के साथ काम करते हुए, अब हम इसके साथ इतना परिचित हो गए हैं कि हमें यह एहसास हो गया है कि इसमें एक बाइनरी श्रेणी है जिस पर हम काम कर सकते हैं: `Color`।

आइए एक लॉजिस्टिक रिग्रेशन मॉडल बनाएं ताकि यह भविष्यवाणी की जा सके कि दिए गए कुछ वेरिएबल्स के आधार पर *किसी दिए गए कद्दू का रंग क्या होने की संभावना है* (नारंगी 🎃 या सफेद 👻)।

> हम रिग्रेशन के बारे में एक पाठ समूह में बाइनरी वर्गीकरण की बात क्यों कर रहे हैं? केवल भाषाई सुविधा के लिए, क्योंकि लॉजिस्टिक रिग्रेशन [वास्तव में एक वर्गीकरण विधि](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) है, हालांकि यह एक लीनियर-आधारित है। डेटा को वर्गीकृत करने के अन्य तरीकों के बारे में अगले पाठ समूह में जानें।

इस पाठ के लिए, हमें निम्नलिखित पैकेजों की आवश्यकता होगी:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) एक [R पैकेजों का संग्रह](https://www.tidyverse.org/packages) है, जिसे डेटा साइंस को तेज़, आसान और अधिक मज़ेदार बनाने के लिए डिज़ाइन किया गया है!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) फ्रेमवर्क एक [पैकेजों का संग्रह](https://www.tidymodels.org/packages/) है, जो मॉडलिंग और मशीन लर्निंग के लिए है।

-   `janitor`: [janitor पैकेज](https://github.com/sfirke/janitor) गंदे डेटा की जांच और सफाई के लिए सरल छोटे टूल प्रदान करता है।

-   `ggbeeswarm`: [ggbeeswarm पैकेज](https://github.com/eclarke/ggbeeswarm) ggplot2 का उपयोग करके बीस्वॉर्म-शैली के प्लॉट बनाने के तरीके प्रदान करता है।

आप इन्हें इस प्रकार इंस्टॉल कर सकते हैं:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

वैकल्पिक रूप से, नीचे दिया गया स्क्रिप्ट यह जांचता है कि क्या आपके पास इस मॉड्यूल को पूरा करने के लिए आवश्यक पैकेज हैं और यदि वे गायब हैं तो उन्हें आपके लिए इंस्टॉल करता है।


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **प्रश्न को परिभाषित करें**

हमारे उद्देश्य के लिए, हम इसे एक बाइनरी के रूप में व्यक्त करेंगे: 'सफेद' या 'सफेद नहीं'। हमारे डेटा सेट में 'धारीदार' श्रेणी भी है, लेकिन इसके उदाहरण बहुत कम हैं, इसलिए हम इसका उपयोग नहीं करेंगे। वैसे भी, जब हम डेटा सेट से null मानों को हटा देते हैं, तो यह गायब हो जाती है।

> 🎃 मज़ेदार तथ्य, हम कभी-कभी सफेद कद्दुओं को 'भूत' कद्दू कहते हैं। इन्हें तराशना आसान नहीं होता, इसलिए ये नारंगी कद्दुओं जितने लोकप्रिय नहीं हैं, लेकिन ये देखने में बहुत अच्छे लगते हैं! तो हम अपने प्रश्न को इस तरह भी दोबारा परिभाषित कर सकते हैं: 'भूत' या 'भूत नहीं'। 👻

## **लॉजिस्टिक रिग्रेशन के बारे में**

लॉजिस्टिक रिग्रेशन कुछ महत्वपूर्ण तरीकों से लीनियर रिग्रेशन से अलग है, जिसके बारे में आपने पहले सीखा था।

#### **बाइनरी वर्गीकरण**

लॉजिस्टिक रिग्रेशन लीनियर रिग्रेशन जैसी विशेषताएं प्रदान नहीं करता। लॉजिस्टिक रिग्रेशन `बाइनरी श्रेणी` ("नारंगी या नारंगी नहीं") के बारे में भविष्यवाणी करता है, जबकि लीनियर रिग्रेशन `सतत मानों` की भविष्यवाणी करने में सक्षम है, जैसे कि कद्दू की उत्पत्ति और कटाई के समय को देखते हुए *उसकी कीमत कितनी बढ़ेगी*।

![दासानी मदीपल्ली द्वारा इन्फोग्राफिक](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.hi.png)

### अन्य वर्गीकरण

लॉजिस्टिक रिग्रेशन के अन्य प्रकार भी हैं, जिनमें मल्टीनोमियल और ऑर्डिनल शामिल हैं:

- **मल्टीनोमियल**, जिसमें एक से अधिक श्रेणियां होती हैं - "नारंगी, सफेद, और धारीदार"।

- **ऑर्डिनल**, जिसमें क्रमबद्ध श्रेणियां होती हैं, जो उपयोगी होती हैं यदि हम अपने परिणामों को तार्किक रूप से क्रमबद्ध करना चाहते हैं, जैसे हमारे कद्दू जो आकारों की एक सीमित संख्या (मिनी, छोटा, मध्यम, बड़ा, एक्सएल, एक्सएक्सएल) के अनुसार क्रमबद्ध होते हैं।

![मल्टीनोमियल बनाम ऑर्डिनल रिग्रेशन](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.hi.png)

#### **चर का सहसंबंध होना आवश्यक नहीं है**

याद है कि लीनियर रिग्रेशन अधिक सहसंबद्ध चर के साथ बेहतर काम करता था? लॉजिस्टिक रिग्रेशन इसके विपरीत है - चर का सहसंबद्ध होना आवश्यक नहीं है। यह इस डेटा के लिए काम करता है, जिसमें सहसंबंध अपेक्षाकृत कमजोर हैं।

#### **आपको बहुत सारा साफ डेटा चाहिए**

लॉजिस्टिक रिग्रेशन अधिक सटीक परिणाम देगा यदि आप अधिक डेटा का उपयोग करते हैं; हमारा छोटा डेटा सेट इस कार्य के लिए आदर्श नहीं है, इसलिए इसे ध्यान में रखें।

✅ उन डेटा प्रकारों के बारे में सोचें जो लॉजिस्टिक रिग्रेशन के लिए उपयुक्त हो सकते हैं।

## अभ्यास - डेटा को व्यवस्थित करें

सबसे पहले, डेटा को थोड़ा साफ करें, null मानों को हटाएं और केवल कुछ कॉलम चुनें:

1. निम्नलिखित कोड जोड़ें:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


आप अपने नए डेटा फ्रेम पर एक नज़र डाल सकते हैं, नीचे दिए गए तरीके से [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) फ़ंक्शन का उपयोग करके:


In [ ]:
pumpkins_select %>% 
  glimpse()


आइए पुष्टि करें कि हम वास्तव में एक बाइनरी वर्गीकरण समस्या कर रहे हैं:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### विज़ुअलाइज़ेशन - श्रेणीबद्ध प्लॉट
अब तक आपने कद्दू डेटा को फिर से लोड किया है और इसे साफ किया है ताकि कुछ वेरिएबल्स सहित एक डेटासेट संरक्षित किया जा सके, जिसमें रंग (Color) शामिल है। चलिए ggplot लाइब्रेरी का उपयोग करके नोटबुक में डेटा फ्रेम को विज़ुअलाइज़ करते हैं।

ggplot लाइब्रेरी आपके डेटा को विज़ुअलाइज़ करने के लिए कुछ शानदार तरीके प्रदान करती है। उदाहरण के लिए, आप श्रेणीबद्ध प्लॉट में प्रत्येक Variety और Color के लिए डेटा के वितरण की तुलना कर सकते हैं।

1. geombar फ़ंक्शन का उपयोग करके ऐसा प्लॉट बनाएं, हमारे कद्दू डेटा का उपयोग करें, और प्रत्येक कद्दू श्रेणी (नारंगी या सफेद) के लिए रंग मैपिंग निर्दिष्ट करें:


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

डेटा का अवलोकन करके, आप देख सकते हैं कि रंग डेटा किस प्रकार वैरायटी से संबंधित है।

✅ इस श्रेणीबद्ध प्लॉट को देखते हुए, आप कौन-कौन से रोचक अन्वेषणों की कल्पना कर सकते हैं?


### डेटा पूर्व-प्रसंस्करण: फीचर एन्कोडिंग

हमारे कद्दू के डेटासेट में सभी कॉलम के लिए स्ट्रिंग मान हैं। श्रेणीबद्ध डेटा के साथ काम करना इंसानों के लिए सहज होता है, लेकिन मशीनों के लिए नहीं। मशीन लर्निंग एल्गोरिदम संख्याओं के साथ बेहतर काम करते हैं। यही कारण है कि डेटा पूर्व-प्रसंस्करण चरण में एन्कोडिंग एक बहुत महत्वपूर्ण कदम है, क्योंकि यह हमें श्रेणीबद्ध डेटा को संख्यात्मक डेटा में बदलने में सक्षम बनाता है, बिना किसी जानकारी को खोए। अच्छी एन्कोडिंग एक अच्छे मॉडल के निर्माण में मदद करती है।

फीचर एन्कोडिंग के लिए मुख्य रूप से दो प्रकार के एन्कोडर होते हैं:

1. **ऑर्डिनल एन्कोडर**: यह ऑर्डिनल वेरिएबल्स के लिए उपयुक्त है, जो श्रेणीबद्ध वेरिएबल्स होते हैं और जिनका डेटा एक तार्किक क्रम का पालन करता है, जैसे हमारे डेटासेट में `item_size` कॉलम। यह एक मैपिंग बनाता है, जिसमें प्रत्येक श्रेणी को एक संख्या द्वारा दर्शाया जाता है, जो उस कॉलम में श्रेणी के क्रम को दर्शाती है।

2. **श्रेणीबद्ध एन्कोडर (Categorical encoder)**: यह नोमिनल वेरिएबल्स के लिए उपयुक्त है, जो श्रेणीबद्ध वेरिएबल्स होते हैं और जिनका डेटा किसी तार्किक क्रम का पालन नहीं करता, जैसे हमारे डेटासेट में `item_size` के अलावा अन्य सभी फीचर्स। यह एक वन-हॉट एन्कोडिंग है, जिसका मतलब है कि प्रत्येक श्रेणी को एक बाइनरी कॉलम द्वारा दर्शाया जाता है: एन्कोडेड वेरिएबल 1 के बराबर होता है यदि कद्दू उस वैरायटी से संबंधित है, और अन्यथा 0 होता है।

Tidymodels एक और शानदार पैकेज प्रदान करता है: [recipes](https://recipes.tidymodels.org/) - डेटा पूर्व-प्रसंस्करण के लिए एक पैकेज। हम एक `recipe` परिभाषित करेंगे, जो यह निर्दिष्ट करता है कि सभी प्रेडिक्टर कॉलम को संख्याओं के सेट में एन्कोड किया जाना चाहिए, इसे `prep` करेंगे ताकि आवश्यक मात्राओं और सांख्यिकी का अनुमान लगाया जा सके, और अंत में `bake` करेंगे ताकि नई डेटा पर गणनाओं को लागू किया जा सके।

> आमतौर पर, recipes का उपयोग मॉडलिंग के लिए एक पूर्व-प्रसंस्करणकर्ता के रूप में किया जाता है, जहां यह परिभाषित करता है कि किसी डेटा सेट पर मॉडलिंग के लिए तैयार करने के लिए कौन-कौन से चरण लागू किए जाने चाहिए। इस मामले में यह **अत्यधिक अनुशंसित** है कि आप `workflow()` का उपयोग करें, बजाय इसके कि मैन्युअली `prep` और `bake` का उपयोग करके recipe का अनुमान लगाएं। हम इसे थोड़ी देर में विस्तार से देखेंगे।
>
> हालांकि, फिलहाल हम recipes + prep + bake का उपयोग यह निर्दिष्ट करने के लिए कर रहे हैं कि डेटा सेट पर कौन-कौन से चरण लागू किए जाने चाहिए ताकि इसे डेटा विश्लेषण के लिए तैयार किया जा सके, और फिर उन चरणों को लागू करके पूर्व-प्रसंस्कृत डेटा को निकाल सकें।


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ आइटम साइज कॉलम के लिए ऑर्डिनल एनकोडर का उपयोग करने के क्या फायदे हैं?

### वेरिएबल्स के बीच संबंधों का विश्लेषण करें

अब जब हमने अपने डेटा को प्री-प्रोसेस कर लिया है, तो हम फीचर्स और लेबल के बीच संबंधों का विश्लेषण कर सकते हैं ताकि यह समझ सकें कि मॉडल फीचर्स के आधार पर लेबल की भविष्यवाणी कितनी अच्छी तरह कर पाएगा। इस प्रकार के विश्लेषण को करने का सबसे अच्छा तरीका डेटा को प्लॉट करना है।  
हम फिर से ggplot की geom_boxplot_ फ़ंक्शन का उपयोग करेंगे, ताकि आइटम साइज, वैरायटी और कलर के बीच संबंधों को एक श्रेणीबद्ध प्लॉट में देखा जा सके। डेटा को बेहतर तरीके से प्लॉट करने के लिए, हम एनकोडेड आइटम साइज कॉलम और अनएनकोडेड वैरायटी कॉलम का उपयोग करेंगे।


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### स्वॉर्म प्लॉट का उपयोग करें

चूंकि Color एक द्विआधारी श्रेणी (सफेद या नहीं) है, इसे विज़ुअलाइज़ेशन के लिए 'एक [विशेष दृष्टिकोण](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) की आवश्यकता होती है।'

Color का वितरण item_size के संदर्भ में दिखाने के लिए `स्वॉर्म प्लॉट` का प्रयास करें।

हम [ggbeeswarm पैकेज](https://github.com/eclarke/ggbeeswarm) का उपयोग करेंगे, जो ggplot2 का उपयोग करके बीस्वॉर्म-शैली के प्लॉट बनाने के तरीके प्रदान करता है। बीस्वॉर्म प्लॉट उन बिंदुओं को प्लॉट करने का एक तरीका है जो आमतौर पर एक-दूसरे के ऊपर आ जाते हैं, ताकि वे एक-दूसरे के बगल में गिरें।


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


अब जब हमें रंग की द्विआधारी श्रेणियों और आकारों के बड़े समूह के बीच संबंध का अंदाजा हो गया है, तो आइए लॉजिस्टिक रिग्रेशन का उपयोग करके किसी दिए गए कद्दू के संभावित रंग का निर्धारण करें।

## अपना मॉडल बनाएं

उन वेरिएबल्स को चुनें जिन्हें आप अपने वर्गीकरण मॉडल में उपयोग करना चाहते हैं और डेटा को प्रशिक्षण और परीक्षण सेट में विभाजित करें। [rsample](https://rsample.tidymodels.org/), जो कि Tidymodels का एक पैकेज है, डेटा को कुशलतापूर्वक विभाजित करने और पुन: नमूना लेने के लिए आधारभूत संरचना प्रदान करता है:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 अब हम एक मॉडल को प्रशिक्षित करने के लिए तैयार हैं, जिसमें प्रशिक्षण फीचर्स को प्रशिक्षण लेबल (रंग) के साथ फिट किया जाएगा।

हम एक रेसिपी बनाकर शुरुआत करेंगे, जो यह निर्दिष्ट करती है कि हमारे डेटा पर मॉडलिंग के लिए तैयार करने के लिए कौन-कौन से प्रीप्रोसेसिंग चरण किए जाने चाहिए, जैसे: श्रेणीबद्ध वेरिएबल्स को संख्याओं में एन्कोड करना। ठीक उसी तरह जैसे `baked_pumpkins`, हम एक `pumpkins_recipe` बनाते हैं लेकिन इसे `prep` और `bake` नहीं करते क्योंकि इसे एक वर्कफ़्लो में शामिल किया जाएगा, जिसे आप कुछ ही चरणों में देखेंगे।

Tidymodels में लॉजिस्टिक रिग्रेशन मॉडल को निर्दिष्ट करने के कई तरीके हैं। देखें `?logistic_reg()`। फिलहाल, हम लॉजिस्टिक रिग्रेशन मॉडल को डिफ़ॉल्ट `stats::glm()` इंजन के माध्यम से निर्दिष्ट करेंगे।


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


अब जब हमारे पास एक रेसिपी और एक मॉडल स्पेसिफिकेशन है, तो हमें इन्हें एक ऐसे ऑब्जेक्ट में जोड़ने का तरीका खोजना होगा, जो पहले डेटा को प्रीप्रोसेस करेगा (पर्दे के पीछे प्रेप+बेक), प्रीप्रोसेस किए गए डेटा पर मॉडल को फिट करेगा, और संभावित पोस्ट-प्रोसेसिंग गतिविधियों की भी अनुमति देगा।

Tidymodels में, यह सुविधाजनक ऑब्जेक्ट [`workflow`](https://workflows.tidymodels.org/) कहलाता है और यह आपके मॉडलिंग घटकों को आसानी से संगठित रखता है।


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


एक बार वर्कफ़्लो *निर्धारित* हो जाने के बाद, मॉडल को [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) फ़ंक्शन का उपयोग करके `प्रशिक्षित` किया जा सकता है। वर्कफ़्लो एक रेसिपी का अनुमान लगाएगा और प्रशिक्षण से पहले डेटा को प्रीप्रोसेस करेगा, इसलिए हमें इसे मैन्युअली prep और bake का उपयोग करके करने की आवश्यकता नहीं होगी।


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


मॉडल प्रिंट आउट प्रशिक्षण के दौरान सीखे गए गुणांक दिखाता है।

अब जब हमने प्रशिक्षण डेटा का उपयोग करके मॉडल को प्रशिक्षित कर लिया है, तो हम [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) का उपयोग करके परीक्षण डेटा पर भविष्यवाणियां कर सकते हैं। चलिए शुरू करते हैं हमारे परीक्षण सेट के लिए लेबल और प्रत्येक लेबल के लिए संभावनाओं की भविष्यवाणी करके। जब संभावना 0.5 से अधिक होती है, तो भविष्यवाणी की गई श्रेणी `WHITE` होती है, अन्यथा `ORANGE`।


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


यह बहुत अच्छा है! यह लॉजिस्टिक रिग्रेशन के काम करने के तरीके के बारे में कुछ और जानकारी प्रदान करता है।

### भ्रमण मैट्रिक्स के माध्यम से बेहतर समझ

प्रत्येक भविष्यवाणी की तुलना उसके संबंधित "ग्राउंड ट्रुथ" वास्तविक मान से करना यह निर्धारित करने का एक बहुत प्रभावी तरीका नहीं है कि मॉडल कितनी अच्छी तरह भविष्यवाणी कर रहा है। सौभाग्य से, Tidymodels के पास कुछ और तरकीबें हैं: [`yardstick`](https://yardstick.tidymodels.org/) - एक पैकेज जो प्रदर्शन मेट्रिक्स का उपयोग करके मॉडलों की प्रभावशीलता को मापने के लिए उपयोग किया जाता है।

वर्गीकरण समस्याओं से संबंधित एक प्रदर्शन मेट्रिक [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix) है। एक भ्रमण मैट्रिक्स यह वर्णन करता है कि वर्गीकरण मॉडल कितना अच्छा प्रदर्शन कर रहा है। भ्रमण मैट्रिक्स यह दिखाता है कि प्रत्येक वर्ग में कितने उदाहरणों को मॉडल द्वारा सही तरीके से वर्गीकृत किया गया। हमारे मामले में, यह आपको दिखाएगा कि कितने नारंगी कद्दू को नारंगी के रूप में वर्गीकृत किया गया और कितने सफेद कद्दू को सफेद के रूप में वर्गीकृत किया गया; भ्रमण मैट्रिक्स यह भी दिखाएगा कि कितने गलत श्रेणियों में वर्गीकृत किए गए।

[`conf_mat()`](https://tidymodels.github.io/yardstick/reference/conf_mat.html) फ़ंक्शन, जो yardstick से आता है, अवलोकित और भविष्यवाणी की गई श्रेणियों का यह क्रॉस-टैबुलेशन गणना करता है।


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


आइए कन्फ्यूजन मैट्रिक्स को समझते हैं। हमारे मॉडल को कद्दू को दो बाइनरी श्रेणियों में वर्गीकृत करने के लिए कहा गया है: श्रेणी `सफेद` और श्रेणी `गैर-सफेद`।

-   यदि आपका मॉडल किसी कद्दू को सफेद के रूप में भविष्यवाणी करता है और वह वास्तव में 'सफेद' श्रेणी में आता है, तो हम इसे `सही सकारात्मक` कहते हैं, जो ऊपर बाईं ओर की संख्या द्वारा दिखाया गया है।

-   यदि आपका मॉडल किसी कद्दू को गैर-सफेद के रूप में भविष्यवाणी करता है और वह वास्तव में 'सफेद' श्रेणी में आता है, तो हम इसे `गलत नकारात्मक` कहते हैं, जो नीचे बाईं ओर की संख्या द्वारा दिखाया गया है।

-   यदि आपका मॉडल किसी कद्दू को सफेद के रूप में भविष्यवाणी करता है और वह वास्तव में 'गैर-सफेद' श्रेणी में आता है, तो हम इसे `गलत सकारात्मक` कहते हैं, जो ऊपर दाईं ओर की संख्या द्वारा दिखाया गया है।

-   यदि आपका मॉडल किसी कद्दू को गैर-सफेद के रूप में भविष्यवाणी करता है और वह वास्तव में 'गैर-सफेद' श्रेणी में आता है, तो हम इसे `सही नकारात्मक` कहते हैं, जो नीचे दाईं ओर की संख्या द्वारा दिखाया गया है।

| सत्यता |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **भविष्यवाणी** | WHITE | ORANGE |
| WHITE        | TP     | FP    |
| ORANGE         | FN     | TN    |

जैसा कि आप अनुमान लगा सकते हैं, यह बेहतर है कि सही सकारात्मक और सही नकारात्मक की संख्या अधिक हो और गलत सकारात्मक और गलत नकारात्मक की संख्या कम हो, क्योंकि इसका मतलब है कि मॉडल बेहतर प्रदर्शन कर रहा है।

कन्फ्यूजन मैट्रिक्स उपयोगी है क्योंकि यह अन्य मेट्रिक्स को जन्म देता है जो हमें वर्गीकरण मॉडल के प्रदर्शन का बेहतर मूल्यांकन करने में मदद कर सकते हैं। आइए उनमें से कुछ को समझते हैं:

🎓 प्रिसीजन: `TP/(TP + FP)` इसे भविष्यवाणी किए गए सकारात्मक परिणामों में से सही सकारात्मक परिणामों का अनुपात कहा जाता है। इसे [सकारात्मक भविष्यवाणी मूल्य](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value") भी कहा जाता है।

🎓 रिकॉल: `TP/(TP + FN)` इसे वास्तविक सकारात्मक नमूनों में से सही सकारात्मक परिणामों का अनुपात कहा जाता है। इसे `संवेदनशीलता` भी कहा जाता है।

🎓 स्पेसिफिसिटी: `TN/(TN + FP)` इसे वास्तविक नकारात्मक नमूनों में से सही नकारात्मक परिणामों का अनुपात कहा जाता है।

🎓 एक्यूरेसी: `TP + TN/(TP + TN + FP + FN)` यह किसी नमूने के लिए सही ढंग से भविष्यवाणी किए गए लेबल का प्रतिशत है।

🎓 एफ माप: प्रिसीजन और रिकॉल का भारित औसत, जिसमें सर्वश्रेष्ठ 1 और सबसे खराब 0 होता है।

आइए इन मेट्रिक्स की गणना करें!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## इस मॉडल के ROC कर्व को विज़ुअलाइज़ करें

आइए एक और विज़ुअलाइज़ेशन करते हैं ताकि तथाकथित [`ROC कर्व`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) को देख सकें:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC वक्र अक्सर किसी वर्गीकरणकर्ता के आउटपुट को उसके सही बनाम गलत सकारात्मक परिणामों के संदर्भ में देखने के लिए उपयोग किए जाते हैं। ROC वक्र आमतौर पर Y अक्ष पर `True Positive Rate`/संवेदनशीलता और X अक्ष पर `False Positive Rate`/1-विशिष्टता को दर्शाते हैं। इसलिए, वक्र की तीव्रता और मध्य रेखा और वक्र के बीच की जगह महत्वपूर्ण होती है: आपको एक ऐसा वक्र चाहिए जो जल्दी से ऊपर और रेखा के पार जाए। हमारे मामले में, शुरुआत में कुछ गलत सकारात्मक परिणाम होते हैं, और फिर रेखा सही तरीके से ऊपर और पार जाती है।

अंत में, चलिए `yardstick::roc_auc()` का उपयोग करके वास्तविक Area Under the Curve की गणना करते हैं। AUC की व्याख्या करने का एक तरीका यह है कि यह उस संभावना को दर्शाता है कि मॉडल एक यादृच्छिक सकारात्मक उदाहरण को एक यादृच्छिक नकारात्मक उदाहरण की तुलना में अधिक रैंक देगा।


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


परिणाम लगभग `0.975` है। चूंकि AUC का मान 0 से 1 के बीच होता है, आप एक बड़ा स्कोर चाहते हैं, क्योंकि यदि कोई मॉडल अपनी भविष्यवाणियों में 100% सही है, तो उसका AUC 1 होगा। इस मामले में, मॉडल *काफी अच्छा* है।

आने वाले पाठों में, जब आप वर्गीकरण (classification) के बारे में और जानेंगे, तो आप यह भी सीखेंगे कि अपने मॉडल के स्कोर को कैसे बेहतर बनाया जाए (जैसे कि इस मामले में असंतुलित डेटा को संभालना)।

## 🚀चुनौती

लॉजिस्टिक रिग्रेशन के बारे में जानने के लिए अभी और भी बहुत कुछ है! लेकिन सीखने का सबसे अच्छा तरीका है प्रयोग करना। ऐसा डेटा सेट खोजें जो इस प्रकार के विश्लेषण के लिए उपयुक्त हो और उसके साथ एक मॉडल बनाएं। आपने क्या सीखा? सुझाव: [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) पर दिलचस्प डेटा सेट्स आज़माएं।

## समीक्षा और स्व-अध्ययन

[स्टैनफोर्ड के इस पेपर](https://web.stanford.edu/~jurafsky/slp3/5.pdf) के शुरुआती कुछ पन्ने पढ़ें, जिसमें लॉजिस्टिक रिग्रेशन के कुछ व्यावहारिक उपयोग बताए गए हैं। उन कार्यों के बारे में सोचें जो अब तक पढ़े गए विभिन्न प्रकार के रिग्रेशन कार्यों के लिए बेहतर उपयुक्त हैं। किस प्रकार का रिग्रेशन सबसे अच्छा काम करेगा?



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।  
